In [ ]:

#@title نصب کتابخانه‌ها
import base64

encoded_text = "Q3JlYXRlIGJ5IDogYWlnb2xkZW4="
decoded_text = base64.b64decode(encoded_text.encode()).decode()
print(decoded_text)
print("="*20)

print("📦 Installing Python libraries...")

!pip install google-genai yt-dlp pysrt pydub youtube-transcript-api git+https://github.com/yaranbarzi/stable-ts.git

print("\n🔧 Installing system tools...")

!sudo apt-get update -y
!sudo apt-get install -y ffmpeg rubberband-cli

print("\n✅ All dependencies installed successfully!")

In [ ]:

#@title آپلود ویدیو از یوتیوب یا حافظه داخلی
#@markdown ### `در صورتی که دانلود از یوتیوب ناموفق بود کوکی های مرورگر را قبل از اجرا پاک کنید`
from google.colab import files
from IPython.display import display
import ipywidgets as widgets
import yt_dlp
import os
import shutil
import glob

# پاکسازی فایل‌های قبلی
for file in glob.glob('input_video*'):
    os.remove(file)
    print(f"فایل ویدیوی قبلی {file} حذف شد.")
if os.path.exists('audio.wav'):
    os.remove('audio.wav')
    print("فایل صوتی قبلی حذف شد.")
if os.path.exists('audio.srt'):
    os.remove('audio.srt')
    print("فایل زیرنویس قبلی حذف شد.")
if os.path.exists('dubbing_project'):
    shutil.rmtree('dubbing_project')
    print("پوشه پروژه دوبله قبلی حذف شد.")

upload_method = "یوتیوب" #@param ["یوتیوب", "حافظه داخلی"]
#@markdown ---
#@markdown #### اگر «یوتیوب» را انتخاب کردید، لینک را اینجا وارد کنید:
YT_Link = "" #@param {type:"string"}
os.environ['YT_Link'] = YT_Link

def process_youtube(url):
    if url.strip():
        for file in glob.glob('temp_video*'):
            if os.path.exists(file):
                os.remove(file)

        # دانلود ویدیو با بالاترین کیفیت
        format_option = 'bestvideo+bestaudio/best'
        temp_filename = 'temp_video.%(ext)s'
        video_opts = {
            'format': format_option,
            'outtmpl': temp_filename,
            'nocheckcertificate': True,
            'ignoreerrors': False,
            'no_warnings': False,
            'quiet': False
        }

        try:
            with yt_dlp.YoutubeDL(video_opts) as ydl:
                info = ydl.extract_info(url, download=True)
                downloaded_file = ydl.prepare_filename(info)

            print(f"فایل دانلود شده: {downloaded_file}")

            if os.path.exists(downloaded_file):
                _, file_extension = os.path.splitext(downloaded_file)
                final_filename = 'input_video' + file_extension
                os.rename(downloaded_file, final_filename)
                print(f"نام فایل به {final_filename} تغییر یافت")

                if file_extension.lower() != '.mp4':
                    os.system(f'ffmpeg -i "{final_filename}" -c copy input_video.mp4 -y')
                    if os.path.exists('input_video.mp4'):
                        os.remove(final_filename)
                        print("فایل به فرمت MP4 تبدیل شد")

                os.system('ffmpeg -i input_video.mp4 -vn audio.wav -y')
                print("صدا از ویدیو استخراج شد.")
                return True
            else:
                print("فایل دانلود شده یافت نشد.")
                return False
        except Exception as e:
            print(f"خطا در دانلود: {str(e)}")
            return False
    return False

if upload_method == "یوتیوب" and YT_Link.strip():
    print("در حال دانلود ویدیو با بالاترین کیفیت...")
    success = process_youtube(YT_Link)
    if success:
        print("ویدیو با موفقیت دانلود شد.")
    else:
        print("دانلود ناموفق بود.")
elif upload_method == "حافظه داخلی":
    print("لطفاً ویدیوی خود را آپلود کنید:")
    uploaded = files.upload()
    video_file = next(iter(uploaded.keys()))
    os.rename(video_file, 'input_video.mp4')
    os.system(f'ffmpeg -i "input_video.mp4" -vn audio.wav -y')
    print("ویدیو آپلود و فایل صوتی استخراج شد.")

if os.path.exists('input_video.mp4'):
    print("\nعملیات آپلود و آماده‌سازی ویدیو با موفقیت انجام شد!")
else:
    print("\nخطا: فایل ویدیویی 'input_video.mp4' ایجاد نشد. لطفاً لینک یا فایل خود را بررسی کنید.")

In [ ]:

#@title ورود و پیکربندی کلید
import google.generativeai as genai
import os

#@markdown ---
#@markdown ### **کلید دسترسی جمینای خود را اینجا وارد کنید**
#@markdown این کلید برای تمام بخش‌های نوتبوک (ترجمه و تولید صدا) استفاده خواهد شد.
GOOGLE_API_KEY = "" #@param {type:"string"}

if not GOOGLE_API_KEY or "YOUR_API_KEY" in GOOGLE_API_KEY:
    print(" هشدار: لطفاً کلید API معتبر خود را وارد کرده و سپس سلول را اجرا کنید.")
else:
    try:
        genai.configure(api_key=GOOGLE_API_KEY)

        os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY
        print(" کلید API با موفقیت برای استفاده در کل نوتبوک پیکربندی شد.")
    except Exception as e:
        print(f" خطا در پیکربندی کلید API: {e}")

In [ ]:
#@title 📝(آپدیت شده) استخراج متن و فشرده‌سازی هوشمند زیرنویس
#@markdown ####فشرده سازی بصورت خودکار انجام میشود
from google.colab import files
import os
import pysrt
import subprocess
import re
import json
import math
from youtube_transcript_api import YouTubeTranscriptApi

#@markdown ---
#@markdown ### **۱. روش استخراج متن**
extraction_method = "Whisper" #@param ["Whisper", "آپلود زیرنویس", "Transcript"]
#@markdown اگر از روش Transcript استفاده می‌کنید، زبان مورد نظر را انتخاب کنید:
transcript_language = "Auto-detect" #@param ["Auto-detect", "English (EN)", "Persian (FA)", "German (DE)", "French (FR)", "Italian (IT)", "Spanish (ES)", "Chinese (ZH)", "Korean (KO)", "Russian (RU)", "Arabic (AR)", "Japanese (JA)", "Hindi (HI)"]

def _parse_srt_for_counting(srt_content):
    subtitle_blocks = srt_content.strip().split('\n\n')
    count = 0
    for block in subtitle_blocks:
        lines = block.strip().split('\n')
        if len(lines) >= 2 and '-->' in lines[1]:
            count += 1
    return count

def compress_srt_dialogues(srt_content_string, merge_count=2):
    def _parse_srt(srt_content):
        subtitle_blocks = srt_content.strip().split('\n\n')
        subtitles = []
        for block in subtitle_blocks:
            lines = block.strip().split('\n')
            if len(lines) >= 2:
                try:
                    time_line_index = -1
                    for i, line in enumerate(lines):
                        if '-->' in line:
                            time_line_index = i
                            break
                    if time_line_index != -1:
                        time_match = re.match(r'(\d{2}:\d{2}:\d{2},\d{3})\s*-->\s*(\d{2}:\d{2}:\d{2},\d{3})', lines[time_line_index])
                        start_time, end_time = time_match.groups()
                        text = '\n'.join(lines[time_line_index+1:])
                        subtitles.append({'start': start_time, 'end': end_time, 'text': text})
                except Exception:
                    continue
        return subtitles

    def _merge_subtitles(subtitles, n):
        merged_subs = []
        new_index = 1
        for i in range(0, len(subtitles), n):
            chunk = subtitles[i:i+n]
            if not chunk: continue
            start_time = chunk[0]['start']
            end_time = chunk[-1]['end']
            combined_text = ' '.join([sub['text'].replace('\n', ' ') for sub in chunk])
            merged_subs.append({'index': new_index, 'start': start_time, 'end': end_time, 'text': combined_text})
            new_index += 1
        return merged_subs

    def _format_srt(subtitles):
        srt_output = []
        for sub in subtitles:
            srt_output.append(str(sub['index']))
            srt_output.append(f"{sub['start']} --> {sub['end']}")
            srt_output.append(sub['text'])
            srt_output.append('')
        return '\n'.join(srt_output)

    original_subtitles = _parse_srt(srt_content_string)
    if not original_subtitles:
        print("هشدار: هیچ دیالوگ معتبری در زیرنویس برای فشرده‌سازی یافت نشد.")
        return srt_content_string

    merged_list = _merge_subtitles(original_subtitles, merge_count)
    new_srt_content = _format_srt(merged_list)

    print(f"فشرده‌سازی {merge_count} تایی انجام شد: {len(original_subtitles)} دیالوگ به {len(merged_list)} دیالوگ تبدیل شد.")
    return new_srt_content

print("--- شروع مرحله ۱: استخراج متن ---")
subtitle_file_path = 'audio.srt'

if extraction_method == "Whisper":
    if os.path.exists('audio.wav'):
        !whisper "audio.wav" --model large --output_dir ./ --output_format srt
        if os.path.exists('audio.srt'):
            print("✔️ زیرنویس با Whisper با موفقیت استخراج شد.")
        else:
            print("❌ خطا در استخراج با Whisper.")
    else:
        print("لطفاً ابتدا یک فایل صوتی آپلود کنید.")

elif extraction_method == "Transcript":
    print("در حال استخراج زیرنویس از یوتیوب...")
    if os.path.exists(subtitle_file_path):
        print(f"✔️ زیرنویس با موفقیت از یوتیوب استخراج و در فایل {subtitle_file_path} ذخیره شد.")
    else:
        print("❌ استخراج زیرنویس از یوتیوب ناموفق بود.")

else:
    print("لطفاً فایل زیرنویس خود را با فرمت .srt آپلود کنید:")
    uploaded = files.upload()
    if uploaded:
        subtitle_file = next(iter(uploaded.keys()))
        os.rename(subtitle_file, subtitle_file_path)
        print(f"✔️ فایل '{subtitle_file}' آپلود و به '{subtitle_file_path}' تغییر نام داد.")
    else:
        print("هیچ فایلی آپلود نشد.")

print("\n--- شروع مرحله ۲: بررسی و فشرده‌سازی هوشمند ---")
if os.path.exists(subtitle_file_path):
    with open(subtitle_file_path, 'r', encoding='utf-8') as f:
        original_content = f.read()

    num_dialogues = _parse_srt_for_counting(original_content)
    print(f"تعداد دیالوگ‌های استخراج شده: {num_dialogues}")

    if num_dialogues > 13:
        merge_n = math.ceil(num_dialogues / 13)
        print(f"تعداد دیالوگ‌ها بیشتر از ۱۳ است. نیاز به فشرده‌سازی با مقدار ادغام {merge_n} می‌باشد.")

        compressed_content = compress_srt_dialogues(original_content, merge_count=merge_n)

        with open(subtitle_file_path, 'w', encoding='utf-8') as f:
            f.write(compressed_content)
        print(f"✔️ فایل زیرنویس با موفقیت فشرده و جایگزین شد.")

    else:
        print("✅ تعداد دیالوگ‌ها ۱۳ یا کمتر است. نیازی به فشرده‌سازی نیست.")
else:
    print(f"⚠️ فایل زیرنویس '{subtitle_file_path}' برای انجام فشرده‌سازی یافت نشد.")

In [ ]:

#@title ترجمه زیرنویس
#@markdown زبان مبدا بصورت خودکار انتخاب میشود و زبان مقصد را دستی انتخاب کنید

import pysrt
import google.generativeai as genai
from tqdm.notebook import tqdm
import time
from google.colab import files
import os
import subprocess
import json

# بررسی اینکه آیا کلید API در سلول قبل تنظیم شده است یا نه
if 'GOOGLE_API_KEY' not in os.environ or not os.environ['GOOGLE_API_KEY']:
     print("❌ خطا: کلید API تنظیم نشده است. لطفاً ابتدا سلول قبلی (ورود کلید API) را اجرا کنید.")
else:
    #@markdown ---
    translation_method = "هوش مصنوعی" #@param ["هوش مصنوعی", "آپلود زیرنویس بصورت دستی"]
    #@markdown زبان مبدا
    source_language = "Auto-detect" #@param ["Auto-detect", "English (EN)", "Persian (FA)", "German (DE)", "French (FR)", "Italian (IT)", "Spanish (ES)", "Chinese (ZH)", "Korean (KO)", "Russian (RU)", "Arabic (AR)", "Japanese (JA)", "Hindi (HI)"]
    #@markdown زبان مقصد
    target_language = "Persian (FA)" #@param ["Persian (FA)", "English (EN)", "German (DE)", "French (FR)", "Italian (IT)", "Spanish (ES)", "Chinese (ZH)", "Korean (KO)", "Russian (RU)", "Arabic (AR)", "Japanese (JA)", "Hindi (HI)"]

    if translation_method == "هوش مصنوعی":
        filename = '/content/audio.srt'
        output_filename = '/content/audio_translated.srt'

        # تعریف لیست مدل‌ها با اولویت مد نظر شما
        translation_models = [
            "gemini-flash-lite-latest",
            "gemini-2.5-flash-lite",
            "gemini-2.5-flash"
        ]

        def translate_with_fallback(text, source_lang=None):
            """
            Tries to translate text using a list of models.
            If one model fails, it automatically tries the next one.
            """
            for model_name in translation_models:
                try:
                    # print(f"Trying model: {model_name}...") # for debugging
                    model = genai.GenerativeModel(
                        model_name,
                        safety_settings={
                            genai.types.HarmCategory.HARM_CATEGORY_HARASSMENT: genai.types.HarmBlockThreshold.BLOCK_NONE,
                            genai.types.HarmCategory.HARM_CATEGORY_HATE_SPEECH: genai.types.HarmBlockThreshold.BLOCK_NONE,
                            genai.types.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: genai.types.HarmBlockThreshold.BLOCK_NONE,
                            genai.types.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: genai.types.HarmBlockThreshold.BLOCK_NONE,
                        }
                    )

                    # --- پرامپت بهینه شده ---
                    if target_language == "Persian (FA)":
                        prompt = f"""
                        شما یک مترجم حرفه‌ای زیرنویس هستید. وظیفه شما فقط ترجمه متنی است که داخل تگ <text> قرار دارد به فارسی عامیانه و روان است.
                        پاسخ شما باید **فقط و فقط** شامل متن ترجمه شده باشد. هیچ‌گونه توضیح، مقدمه، یا کلمه اضافه‌ای ننویسید.

                        مثال:
                        ورودی: <text>This is a test.</text>
                        خروجی: این یک آزمایش است.

                        حالا متن زیر را ترجمه کن:
                        ورودی: <text>{text}</text>
                        خروجی:
                        """
                    else:
                        language_map = {"English (EN)": "English", "German (DE)": "German", "French (FR)": "French", "Italian (IT)": "Italian", "Spanish (ES)": "Spanish", "Chinese (ZH)": "Chinese", "Korean (KO)": "Korean", "Russian (RU)": "Russian", "Arabic (AR)": "Arabic", "Japanese (JA)": "Japanese", "Hindi (HI)": "Hindi"}
                        target_lang_name = language_map.get(target_language, "English")
                        prompt = f"""
                        You are an expert subtitle translator. Your only task is to translate the text inside the <text> tag to {target_lang_name}.
                        Your response must ONLY be the translated text. Do not add any explanation, prefix, or extra words.

                        Example:
                        Input: <text>سلام، این یک آزمایش است.</text>
                        Output: Hello, this is a test.

                        Now, translate the following:
                        Input: <text>{text}</text>
                        Output:
                        """
                    # --- پایان پرامپت ---

                    response = model.generate_content(prompt)
                    time.sleep(2)  # To avoid rate limiting
                    cleaned_text = response.text.strip()

                    # اگر ترجمه موفق بود، متن را برگردان و از حلقه خارج شو
                    return cleaned_text

                except Exception as e:
                    print(f"⚠️ مدل {model_name} با خطا مواجه شد: {str(e)}. در حال تلاش با مدل بعدی...")
                    time.sleep(3) # A short pause before trying the next model

            # اگر تمام مدل‌ها شکست خوردند، متن اصلی را برگردان تا کار متوقف نشود
            print(f"❌ تمام مدل‌های ترجمه برای قطعه '{text[:30]}...' ناموفق بودند. متن اصلی جایگزین شد.")
            return text

        try:
            subs = pysrt.open(filename)
            with tqdm(total=len(subs), desc="ترجمه زیرنویس") as pbar:
                for i, sub in enumerate(subs):
                    sub.text = translate_with_fallback(sub.text, source_language if source_language != "Auto-detect" else None)
                    pbar.update(1)
            subs.save(output_filename, encoding='utf-8')
            os.rename(output_filename, 'audio_fa.srt')
            print(f"\nترجمه از {source_language} به {target_language} با موفقیت به پایان رسید!")

        except Exception as e:
            print(f"خطا در خواندن یا ترجمه فایل زیرنویس: {str(e)}")

    else:  # آپلود زیرنویس بصورت دستی
        print("لطفاً فایل زیرنویس ترجمه شده خود را با فرمت .srt آپلود کنید:")
        uploaded = files.upload()
        subtitle_file = next(iter(uploaded.keys()))
        os.rename(subtitle_file, 'audio_fa.srt')

    if os.path.exists('audio_fa.srt'):
        print("\n✅ فایل زیرنویس ترجمه شده 'audio_fa.srt' آماده است.")
    else:
        print("\n❌ خطا: فایل زیرنویس ترجمه شده 'audio_fa.srt' ایجاد نشد.")

In [ ]:

#@title ساخت سگمنت‌های صوتی
import google.genai as genai
from google.genai import types
import pysrt
import os
import subprocess
import traceback
from pydub import AudioSegment
import time
import shutil
import mimetypes
import struct

#@markdown ---
#@markdown ### **تنظیمات زمان استراحت**
#@markdown برای جلوگیری از خطا، می‌توانید زمان انتظار بین درخواست‌ها را افزایش دهید.

#@markdown زمان استراحت بین هر درخواست موفق (ثانیه):
sleep_between_requests = 9 #@param {type:"slider", min:3, max:20, step:1}

#@markdown زمان استراحت پایه بین تلاش‌های مجدد پس از خطا (ثانیه):
sleep_between_retries = 9 #@param {type:"slider", min:5, max:30, step:1}

#@markdown ---
#@markdown ### **پرامپت برای تعیین لحن صدا**
#@markdown در اینجا می‌توانید لحن صدا را مشخص کنید (مثلا: با لحنی آرام و شمرده صحبت کن)
speech_prompt = "" #@param {type:"string"}

#@markdown ---
#@markdown ### **تنظیمات تولید صدا با مدل‌های اختصاصی TTS**
#@markdown نسخه فلش رایگان میباشد
tts_model_name = "gemini-2.5-flash-preview-tts" #@param ["gemini-2.5-flash-preview-tts", "gemini-2.5-pro-preview-tts"]

#@markdown ---
#@markdown ### **انتخاب گوینده**
#@markdown لیست کامل گویندگان حرفه‌ای Gemini:
speaker_voice = "Fenrir" #@param ["Achird", "Zubenelgenubi", "Vindemiatrix", "Sadachbia", "Sadaltager", "Sulafat", "Laomedeia", "Achernar", "Alnilam", "Schedar", "Gacrux", "Pulcherrima", "Umbriel", "Algieba", "Despina", "Erinome", "Algenib", "Rasalthgeti", "Orus", "Aoede", "Callirrhoe", "Autonoe", "Enceladus", "Iapetus", "Zephyr", "Puck", "Charon", "Kore", "Fenrir", "Leda"]


os.makedirs('dubbing_project/dubbed_segments', exist_ok=True)


def parse_audio_mime_type(mime_type: str) -> dict:
    parts = mime_type.split(";")
    details = {'bits_per_sample': 16, 'rate': 24000}
    for param in parts:
        param = param.strip()
        if param.lower().startswith("rate="):
            details['rate'] = int(param.split("=", 1)[1])
        elif param.startswith("audio/L"):
            details['bits_per_sample'] = int(param.split("L", 1)[1])
    return details

def convert_to_wav(audio_data: bytes, mime_type: str) -> bytes:
    parameters = parse_audio_mime_type(mime_type)
    bits_per_sample = parameters["bits_per_sample"]
    sample_rate = parameters["rate"]
    num_channels = 1
    data_size = len(audio_data)
    bytes_per_sample = bits_per_sample // 8
    block_align = num_channels * bytes_per_sample
    byte_rate = sample_rate * block_align
    chunk_size = 36 + data_size
    header = struct.pack(
        "<4sI4s4sIHHIIHH4sI",
        b"RIFF", chunk_size, b"WAVE", b"fmt ", 16, 1,
        num_channels, sample_rate, byte_rate, block_align,
        bits_per_sample, b"data", data_size
    )
    return header + audio_data



def generate_gemini_tts_segment(client, text, prompt, voice, model, output_path, max_retries=3):
    for attempt in range(1, max_retries + 1):
        try:
            if prompt and prompt.strip():
                final_text = f"{prompt.strip()}: \"{text}\""
            else:
                final_text = text

            contents = [types.Content(role="user", parts=[types.Part.from_text(text=final_text)])]
            generate_content_config = types.GenerateContentConfig(
                response_modalities=["audio"],
                speech_config=types.SpeechConfig(
                    voice_config=types.VoiceConfig(
                        prebuilt_voice_config=types.PrebuiltVoiceConfig(voice_name=voice)
                    )
                ),
            )
            stream = client.models.generate_content_stream(
                model=model, contents=contents, config=generate_content_config,
            )

            audio_data_buffer = b""
            mime_type = ""
            for chunk in stream:
                if chunk.candidates and chunk.candidates[0].content and chunk.candidates[0].content.parts:
                    part = chunk.candidates[0].content.parts[0]
                    if part.inline_data:
                        audio_data_buffer += part.inline_data.data
                        mime_type = part.inline_data.mime_type

            if audio_data_buffer and mime_type:
                final_wav_data = convert_to_wav(audio_data_buffer, mime_type)
                with open(output_path, 'wb') as f:
                    f.write(final_wav_data)
                return output_path
            else:
                raise Exception("هیچ داده صوتی از API دریافت نشد.")

        except Exception as e:
            print(f"❌ خطا در تولید صدای Gemini (تلاش {attempt}/{max_retries}): {str(e)}")
            if attempt < max_retries:

                wait_time = sleep_between_retries * attempt
                print(f"⏳ انتظار برای {wait_time} ثانیه قبل از تلاش مجدد...")
                time.sleep(wait_time)
            else:
                print(f"💔 تولید صدا برای قطعه '{text[:50]}...' ناموفق بود.")
                return None
    return None


def run_dubbing_process():
    if 'GOOGLE_API_KEY' not in globals() or not GOOGLE_API_KEY:
        print("❌ کلید API جمینای در سلول قبلی وارد نشده است. لطفاً ابتدا سلول ترجمه را اجرا کنید.")
        return

    try:
        client = genai.Client(api_key=GOOGLE_API_KEY)
        print("✅ کلاینت Gemini با موفقیت ایجاد شد.")
    except Exception as e:
        print(f"❌ خطا در ایجاد کلاینت Gemini: {e}. لطفاً کلید API خود را بررسی کنید.")
        return

    try:
        subs = pysrt.open('/content/audio_fa.srt', encoding='utf-8')
    except Exception as e:
        print(f"❌ خطا در خواندن فایل زیرنویس 'audio_fa.srt': {str(e)}")
        return

    print(f"🚀 شروع تولید {len(subs)} سگمنت صوتی با مدل {tts_model_name} و صدای {speaker_voice}...")
    if speech_prompt and speech_prompt.strip():
        print(f"🗣️ با استفاده از پرامپت لحن: '{speech_prompt}'")
    print("="*50)

    for i, sub in enumerate(subs):
        print(f"🎧 پردازش سگمنت {i+1}/{len(subs)}...")
        temp_audio_path = f"dubbing_project/dubbed_segments/temp_{i+1}.wav"
        final_segment_path = f"dubbing_project/dubbed_segments/dub_{i+1}.wav"

        generated_path = generate_gemini_tts_segment(client, sub.text, speech_prompt, speaker_voice, tts_model_name, temp_audio_path)


        if i < len(subs) - 1:
            print(f"⏱️ استراحت برای {sleep_between_requests} ثانیه...")
            time.sleep(sleep_between_requests)

        if not generated_path or not os.path.exists(generated_path):
            print(f"⚠️ تولید صدای Gemini برای سگمنت {i+1} ناموفق بود. یک فایل سکوت ایجاد می‌شود.")
            start_ms = sub.start.hours * 3600000 + sub.start.minutes * 60000 + sub.start.seconds * 1000 + sub.start.milliseconds
            end_ms = sub.end.hours * 3600000 + sub.end.minutes * 60000 + sub.end.seconds * 1000 + sub.end.milliseconds
            target_duration_ms = end_ms - start_ms
            silent_audio = AudioSegment.silent(duration=max(int(target_duration_ms), 100))
            silent_audio.export(final_segment_path, format="wav")
            continue

        try:
            start_ms = sub.start.hours * 3600000 + sub.start.minutes * 60000 + sub.start.seconds * 1000 + sub.start.milliseconds
            end_ms = sub.end.hours * 3600000 + sub.end.minutes * 60000 + sub.end.seconds * 1000 + sub.end.milliseconds
            target_duration = (end_ms - start_ms) / 1000.0
            if target_duration <= 0: target_duration = 0.5

            sound = AudioSegment.from_file(generated_path)
            original_duration = len(sound) / 1000.0

            if original_duration == 0:
                raise ValueError("فایل صوتی تولید شده خالی است.")

            speed_factor = original_duration / target_duration
            speed_factor = max(0.5, min(speed_factor, 2.5))

            print(f"   - زمان هدف: {target_duration:.2f}s | زمان اصلی: {original_duration:.2f}s | ضریب سرعت: {speed_factor:.2f}")

            subprocess.run([
                'ffmpeg', '-i', generated_path,
                '-filter:a', f'rubberband=tempo={speed_factor}',
                '-y', final_segment_path
            ], check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

            print(f"   ✅ سگمنت {i+1} با موفقیت ساخته و زمان‌بندی شد.")

        except Exception as e:
            print(f"   ❌ خطا در زمان‌بندی سگمنت {i+1}: {e}")
            shutil.copy(generated_path, final_segment_path)
        #finally:
            #if os.path.exists(generated_path):
                #os.remove(generated_path)

    print("="*50)
    print("🎉 تمام سگمنت‌های صوتی با Gemini TTS با موفقیت ساخته شدند!")


run_dubbing_process()

In [ ]:

#@title شروع عملیات نهایی
import subprocess
import pysrt
import os
from datetime import timedelta
import json
from pydub import AudioSegment
import tempfile
import glob
import shutil
from IPython.display import display, HTML

video_files = glob.glob('input_video.*')
if not os.path.exists('input_video.mp4') and video_files:
    input_video = video_files[0]
else:
    input_video = 'input_video.mp4'

#@markdown ---
#@markdown ### **تنظیمات صدای اصلی**
#@markdown آیا می‌خواهید صدای اصلی ویدیو حفظ شود؟
keep_original_audio = False #@param {type:"boolean"}
#@markdown میزان صدای اصلی ویدیو (فقط در صورت فعال بودن گزینه بالا)
original_audio_volume = 0.8 #@param {type:"slider", min:0, max:1, step:0.005}

#@markdown ---
#@markdown ### **روش ترکیب صدا**

audio_merge_method = "pydub" #@param ["pydub", "ffmpeg filter_complex", "ffmpeg concat"]


if not os.path.exists(input_video):
    print(f"❌ خطا: فایل ویدیویی {input_video} یافت نشد!")
else:
    print(f"✔️ فایل ویدیو یافت شد: {input_video}")

if not os.path.exists('/content/audio_fa.srt'):
    print("❌ خطا: فایل زیرنویس 'audio_fa.srt' یافت نشد!")
else:
    print("✔️ فایل زیرنویس یافت شد")

segment_dir = "dubbing_project/dubbed_segments"
if not os.path.exists(segment_dir):
    print("❌ خطا: پوشه سگمنت‌های صوتی یافت نشد!")
    os.makedirs(segment_dir, exist_ok=True)
else:
    segments = [f for f in os.listdir(segment_dir) if f.startswith("dub_") and f.endswith(".wav")]
    print(f"✔️ {len(segments)} سگمنت صوتی یافت شد")

# --- خواندن فایل زیرنویس با بررسی فرمت‌های مختلف ---
try:
    subs = pysrt.open('/content/audio_fa.srt', encoding='utf-8')
    print("✔️ فایل زیرنویس با موفقیت خوانده شد (utf-8)")
except Exception:
    try:
        subs = pysrt.open('/content/audio_fa.srt', encoding='latin-1')
        print("✔️ فایل زیرنویس با موفقیت خوانده شد (latin-1)")
    except Exception:
        try:
            print("⚠️ در حال تلاش برای تبدیل فرمت زیرنویس به UTF-8...")
            subprocess.run(['iconv', '-f', 'ISO-8859-1', '-t', 'UTF-8', '/content/audio_fa.srt', '-o', '/content/audio_fa_utf8.srt'], check=True)
            subs = pysrt.open('/content/audio_fa_utf8.srt')
            print("✔️ فایل زیرنویس با تبدیل به UTF-8 خوانده شد")
        except Exception as e:
            print(f"❌ خطا در خواندن فایل زیرنویس حتی پس از تبدیل فرمت: {str(e)}")


# --- تعیین نام فایل خروجی ---
try:
    voice_code = speaker_voice.split("(")[1].split(")")[0] if "(" in speaker_voice else "FA"
    output_filename = f'final_dubbed_video_{voice_code}.mp4'
    if os.path.exists(output_filename):
        os.remove(output_filename)
        print(f"✔️ فایل خروجی قبلی '{output_filename}' حذف شد")
except NameError:
    print("⚠️ متغیر 'speaker_voice' تعریف نشده است. لطفاً سلول ساخت صدا را اجرا کنید.")
    output_filename = 'final_dubbed_video.mp4'


# --- شروع فرآیند ترکیب صدا ---
temp_dir = None
try:
    temp_dir = tempfile.mkdtemp()

    # روش 1: استفاده از فایل concat
    if audio_merge_method == "ffmpeg concat":
        print("⚙️ در حال استفاده از روش ffmpeg concat...")

        segment_info_file = os.path.join(temp_dir, "segments.txt")
        original_audio_path = os.path.join(temp_dir, "original_audio.wav")
        subprocess.run(['ffmpeg', '-i', input_video, '-vn', '-acodec', 'pcm_s16le', '-ar', '44100', '-ac', '2', '-y', original_audio_path], check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        modified_original_audio = os.path.join(temp_dir, "original_audio_modified.wav")
        volume = original_audio_volume if keep_original_audio else 0.0
        subprocess.run(['ffmpeg', '-i', original_audio_path, '-filter:a', f'volume={volume}', '-y', modified_original_audio], check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        with open(segment_info_file, 'w') as f:
            for i, sub in enumerate(subs):
                start_time_ms = (sub.start.hours * 3600 + sub.start.minutes * 60 + sub.start.seconds) * 1000 + sub.start.milliseconds
                start_time_sec = start_time_ms / 1000.0
                segment_path = f"dubbing_project/dubbed_segments/dub_{i+1}.wav"
                if os.path.exists(segment_path):
                    f.write(f"file '{os.path.abspath(segment_path)}'\n")
        merged_audio = os.path.join(temp_dir, "merged_audio.wav")
        dubbing_audio = os.path.join(temp_dir, "dubbing_audio.wav")
        subprocess.run(['ffmpeg', '-f', 'concat', '-safe', '0', '-i', segment_info_file, '-c', 'copy', '-y', dubbing_audio], check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        subprocess.run(['ffmpeg', '-i', modified_original_audio, '-i', dubbing_audio, '-filter_complex', '[0:a][1:a]amix=inputs=2:duration=longest', '-y', merged_audio], check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        subprocess.run(['ffmpeg', '-i', input_video, '-i', merged_audio, '-c:v', 'copy', '-c:a', 'aac', '-map', '0:v', '-map', '1:a', '-y', output_filename], check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        print(f"🎉 ویدیو با موفقیت با روش concat ساخته شد: {output_filename}")
        from google.colab import files
        files.download(output_filename)


    # روش 2: استفاده از filter_complex
    elif audio_merge_method == "ffmpeg filter_complex":
        print("⚙️ در حال استفاده از روش ffmpeg filter_complex...")

        if keep_original_audio:
            filter_complex = f"[0:a]volume={original_audio_volume}[original_audio];"
        else:
            filter_complex = "[0:a]volume=0[original_audio];"
        valid_segments = []
        input_files_list = ['-i', input_video]
        for i, sub in enumerate(subs):
            segment_path = f"dubbing_project/dubbed_segments/dub_{i+1}.wav"
            if os.path.exists(segment_path):
                try:
                    start_time_ms = (sub.start.hours * 3600 + sub.start.minutes * 60 + sub.start.seconds) * 1000 + sub.start.milliseconds
                    if start_time_ms < 0: start_time_ms = 0
                    filter_complex += f"[{i+1}:a]adelay={start_time_ms}|{start_time_ms}[a{i+1}];"
                    valid_segments.append(i+1)
                    input_files_list.append('-i')
                    input_files_list.append(segment_path)
                except Exception as e:
                    print(f"⚠️ رد کردن سگمنت {i+1} به دلیل مشکل: {str(e)}")
        if valid_segments:
            merge_command = "[original_audio]"
            for i in valid_segments:
                merge_command += f"[a{i}]"
            merge_command += f"amix=inputs={len(valid_segments) + 1}:normalize=0[aout]"
            filter_complex += merge_command
            command = ['ffmpeg', '-y'] + input_files_list + ['-filter_complex', filter_complex, '-map', '0:v', '-map', '[aout]', '-c:v', 'copy', '-c:a', 'aac', output_filename]
            result = subprocess.run(command, capture_output=True, text=True)
            if result.returncode != 0:
                print("❌ خطای ffmpeg (filter_complex):")
                print(result.stderr)
                raise Exception("فایل خروجی با filter_complex ساخته نشد!")
            else:
                print(f"🎉 ویدیو با موفقیت با روش filter_complex ساخته شد: {output_filename}")
                from google.colab import files
                files.download(output_filename)
        else:
            raise Exception("هیچ سگمنت صوتی معتبری یافت نشد!")

    # روش 3: استفاده از pydub
    elif audio_merge_method == "pydub":
        print("⚙️ در حال استفاده از روش pydub...")
        original_audio_path = os.path.join(temp_dir, "original_audio.wav")
        subprocess.run([
            'ffmpeg', '-i', input_video, '-vn',
            '-acodec', 'pcm_s16le', '-ar', '44100', '-ac', '2',
            '-y', original_audio_path
        ], check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

        original_audio = AudioSegment.from_file(original_audio_path)
        if keep_original_audio:
            volume_reduction = - (60 * (1 - original_audio_volume))
            original_audio = original_audio + volume_reduction
        else:
            original_audio = AudioSegment.silent(duration=len(original_audio))

        final_audio = original_audio
        for i, sub in enumerate(subs):
            segment_path = f"dubbing_project/dubbed_segments/dub_{i+1}.wav"
            if os.path.exists(segment_path):
                try:
                    segment_audio = AudioSegment.from_file(segment_path)
                    start_time_ms = (sub.start.hours * 3600 + sub.start.minutes * 60 + sub.start.seconds) * 1000 + sub.start.milliseconds
                    if start_time_ms < 0: start_time_ms = 0
                    final_audio = final_audio.overlay(segment_audio, position=start_time_ms)
                except Exception as e:
                    print(f"⚠️ خطا در اضافه کردن سگمنت {i+1}: {str(e)}")

        merged_audio_path = os.path.join(temp_dir, "merged_audio.wav")
        final_audio.export(merged_audio_path, format="wav")

        subprocess.run([
            'ffmpeg', '-i', input_video, '-i', merged_audio_path,
            '-c:v', 'copy', '-c:a', 'aac', '-map', '0:v', '-map', '1:a',
            '-y', output_filename
        ], check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

        print(f"\n🎉 ویدیو با موفقیت با روش pydub ساخته شد: {output_filename}")


        image_url = 'https://huggingface.co/Toolsai/dubtest/resolve/main/newgolden.png'
        youtube_channel_url = 'https://youtube.com/@aigolden'
        html_code = f'''
        <div style="text-align: center; border: 2px solid #e0e0e0; padding: 15px; border-radius: 12px; background-color: #f9f9f9; max-width: 350px; margin: auto;">
            <a href="{youtube_channel_url}" target="_blank" title="رفتن به کانال یوتیوب AIGOLDEN">
                <img src="{image_url}" alt="AIGOLDEN YouTube Channel" style="max-width: 100%; height: auto; border-radius: 8px;">
            </a>
            <p style="font-size: 16px; font-family: 'Vazir', sans-serif; margin-top: 15px; color: #333;">
                برای مشاهده آموزش‌های بیشتر، ما را در یوتیوب دنبال کنید.
            </p>
            <a href="{youtube_channel_url}" target="_blank" style="text-decoration: none; display: inline-block; background-color: #FF0000; color: white; padding: 10px 20px; border-radius: 8px; font-weight: bold; font-family: 'Vazir', sans-serif; margin-top: 10px;">
                🚀 دنبال کردن در یوتیوب
            </a>
        </div>
        '''
        display(HTML(html_code))


        print("\n📥 در حال آماده‌سازی فایل برای دانلود...")
        from google.colab import files
        files.download(output_filename)

except Exception as e:
    print(f"❌ یک خطای کلی در فرآیند ترکیب نهایی رخ داد: {str(e)}")
    # Fallback logic for pydub if other methods fail
    if audio_merge_method != 'pydub':
        print("⚠️ خطایی در روش انتخابی رخ داد، در حال امتحان روش pydub...")

finally:
    if temp_dir and os.path.exists(temp_dir):
        shutil.rmtree(temp_dir)
        print("🧹 فایل‌های موقت پاک شدند.")

In [ ]:

#@title دانلود فایل صوتی کامل دوبله (با سرعت عادی)
import os
import glob
from pydub import AudioSegment
from google.colab import files
import re

segments_folder = "dubbing_project/dubbed_segments"
output_audio_filename = "final_normal_speed_dub.mp3"

print(f"⚙️ در حال جستجو برای فایل‌های صوتی در پوشه: {segments_folder}")


temp_files = glob.glob(os.path.join(segments_folder, "temp_*.wav"))

if not temp_files:
    print("❌ هیچ فایل صوتی با سرعت عادی یافت نشد.")
    print("💡 لطفاً ابتدا سلول ۶ (ساخت سگمنت‌های صوتی) را اجرا کنید و مطمئن شوید که خط مربوط به حذف فایل‌های موقت را کامنت کرده‌اید.")
else:
    # مرتب‌سازی عددی فایل‌ها برای اطمینان از ترتیب صحیح (مثلا temp_1, temp_2, ..., temp_10)
    temp_files.sort(key=lambda f: int(re.search(r'temp_(\d+)', f).group(1)))

    print(f"✔️ {len(temp_files)} فایل صوتی با سرعت عادی پیدا شد. در حال ادغام...")

    # ادغام فایل‌ها با استفاده از pydub
    combined_audio = AudioSegment.empty()
    for file_path in temp_files:
        try:
            segment = AudioSegment.from_file(file_path)
            combined_audio += segment
        except Exception as e:
            print(f"⚠️ خطا در خواندن فایل {os.path.basename(file_path)}: {e}")

    # ذخیره فایل نهایی با فرمت MP3 برای حجم کمتر
    try:
        combined_audio.export(output_audio_filename, format="mp3", bitrate="192k")
        print(f"\n🎉 فایل صوتی کامل با نام '{output_audio_filename}' با موفقیت ایجاد شد.")
        print("📥 در حال آماده‌سازی برای دانلود...")
        files.download(output_audio_filename)
    except Exception as e:
        print(f"❌ خطا در ذخیره فایل نهایی: {e}")

In [ ]:
#@title پاکسازی فایل‌های جلسه قبلی
!rm -rf /content/*